# PCAP Request and Retrieval Notebook (Security Onion standalone)

In this notebook, we will:

1. **Load PCAP Request Information**: We'll read a CSV file that contains necessary details about the requests we want to make. This includes source IPs, destination IPs, source ports, and destination ports.
2. **Formulate Google Stenographer Requests**: Based on the loaded information, we will craft the specific stenographer request format.
3. **Establish and Test an SSH Connection**: Before proceeding with our PCAP operations, we need to ensure that our SSH connection to the remote server is reliable. Thus, we'll try establishing a connection to the specified remote server.
4. **Request and Retrieve PCAP Files**: This is the core operation where we'll connect to the remote server, execute our crafted stenographer requests, and then fetch the resultant PCAP files back to our local machine.


## Library Import

In [1]:
import paramiko
import scp
import pandas as pd
import socket

## Load CSV Information

In [2]:
csv_path = "./Desktop/flow_info.csv"  # Replace with the path to your CSV file

pcap_info = pd.read_csv(csv_path)
pcap_info

,log.id.uid,destination.ip,destination.port,source.port,source.ip,network.transport,@timestamp,host.name
0,479970930030821,10.0.2.20,49678,4444,10.0.2.10,TCP,2023-07-31T08:28:58.036Z,wajdi
1,1497695079384002,10.0.2.20,5802,59476,10.0.2.10,TCP,2023-07-30T11:53:20.803Z,wajdi
2,431196717720327,10.0.2.20,1433,46816,10.0.2.10,TCP,2023-07-30T11:53:20.140Z,wajdi
3,2068159078114237,10.0.2.20,5432,41494,10.0.2.10,TCP,2023-07-30T11:53:20.369Z,wajdi
4,1978009862068205,10.0.2.20,1433,46818,10.0.2.10,TCP,2023-07-30T11:53:20.251Z,wajdi
5,1645336727519481,10.0.2.20,5907,36920,10.0.2.10,TCP,2023-07-30T11:53:19.610Z,wajdi
6,2162882434303965,10.0.2.20,1521,44098,10.0.2.10,TCP,2023-07-30T11:53:20.264Z,wajdi
7,1190175420902199,10.0.2.20,3306,56704,10.0.2.10,TCP,2023-07-30T11:53:18.584Z,wajdi
8,138803934140897,10.0.2.20,5432,41484,10.0.2.10,TCP,2023-07-30T11:53:20.268Z,wajdi
9,2065169772096370,10.0.2.20,3306,56682,10.0.2.10,TCP,2023-07-30T11:51:06.568Z,wajdi


## Crafting "Google Stenographer" Requests

In [3]:
pcap_info['request'] = [
    f'"(host {row["source.ip"]}) and '
    f'(port {row["source.port"]}) and '
    f'(host {row["destination.ip"]}) and '
    f'(port {row["destination.port"]})"'
    for _, row in pcap_info.iterrows()
]

for i in pcap_info['request']:
    print(i)

"(host 10.0.2.10) and (port 4444) and (host 10.0.2.20) and (port 49678)"
"(host 10.0.2.10) and (port 59476) and (host 10.0.2.20) and (port 5802)"
"(host 10.0.2.10) and (port 46816) and (host 10.0.2.20) and (port 1433)"
"(host 10.0.2.10) and (port 41494) and (host 10.0.2.20) and (port 5432)"
"(host 10.0.2.10) and (port 46818) and (host 10.0.2.20) and (port 1433)"
"(host 10.0.2.10) and (port 36920) and (host 10.0.2.20) and (port 5907)"
"(host 10.0.2.10) and (port 44098) and (host 10.0.2.20) and (port 1521)"
"(host 10.0.2.10) and (port 56704) and (host 10.0.2.20) and (port 3306)"
"(host 10.0.2.10) and (port 41484) and (host 10.0.2.20) and (port 5432)"
"(host 10.0.2.10) and (port 56682) and (host 10.0.2.20) and (port 3306)"
"(host 10.0.2.10) and (port 41468) and (host 10.0.2.20) and (port 5432)"
"(host 10.0.2.10) and (port 46798) and (host 10.0.2.20) and (port 1433)"
"(host 10.0.2.10) and (port 44090) and (host 10.0.2.20) and (port 1521)"
"(host 10.0.2.10) and (port 42578) and (host 10.0.2

## Setting Up Remote Connection Details
Before we begin our remote operations, we need to specify the details of our remote host. This includes the host IP, username, and password. Make sure these are correctly set before moving on.


In [4]:
# Set up your remote connection details.
remote_host = "192.168.43.174"
remote_username = "wajdi"
remote_password = "0000"

## Testing SSH Connection to Remote Server
Prior to sending requests and retrieving PCAP files, we'll establish a test SSH connection to our remote server. This step ensures that we have the necessary access and that there are no connectivity issues.


In [5]:
def test_ssh_connection(host, username, password):
    """
    Connect via SSH and retrieve a pcap file from the remote server.
    """
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        ssh.connect(host, username=username, password=password)
        print("Connected via SSH")
    except (paramiko.AuthenticationException, paramiko.SSHException) as error:
        print(f"Error: {str(error)}")
    except (socket.timeout, socket.error) as e:
        print(f"Network error while trying to connect to host {host}:")
        print(f"      {str(e)}")
    finally:
        ssh.close()
        
test_ssh_connection(remote_host,remote_username,remote_password)

Connected via SSH


## Requesting and Retrieving PCAP Files from Remote Server

In this step, we perform two main operations for each of our crafted stenographer requests:

1. **Send the Request to the Server**: For each request, we connect to the server via SSH and execute the stenographer command.
2. **Retrieve the Resultant PCAP File**: Once we've sent the request, the remote server will process it and produce a PCAP file. We then retrieve this file back to our local machine.

Both operations come with their own set of error checks to handle any issues that might arise during the process.


In [6]:
# Iterate through each pcap request.
for _, row in pcap_info.iterrows():
    # Set up the SSH client.
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # Try to connect and request the PCAP file.
    try:
        ssh.connect(remote_host, username=remote_username, password=remote_password)
        
        # Requesting the PCAP file.
        command = f"sudo so-pcap-export {row['request']} {row['log.id.uid']}"
        _, stdout, _ = ssh.exec_command(command)
        
        print(f"Executed command: {command}")
        print("Command output:", stdout.read().decode())
        
    except (paramiko.AuthenticationException, paramiko.SSHException) as error:
        print(f"Error while requesting PCAP: {str(error)}")
        
    finally:
        ssh.close()

    # Retrieve the requested PCAP file.
    log_id = row['log.id.uid']
    local_filename = f"./filtred_pcap_{log_id}.pcap"
    remote_pcap_path = f"/nsm/pcapout/{log_id}.pcap"
    
    # Set up another SSH client for retrieving the file.
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    try:
        ssh.connect(remote_host, username=remote_username, password=remote_password)
        
        # Using SCP to get the file.
        with scp.SCPClient(ssh.get_transport()) as scp_client:
            scp_client.get(remote_pcap_path, local_filename)
            
        print(f"PCAP file {local_filename} retrieved successfully.")
        
    except (paramiko.AuthenticationException, paramiko.SSHException) as error:
        print(f"Error while retrieving PCAP: {str(error)}")
        
    finally:
        ssh.close()

print("PCAP request and retrieval process completed.")


Executed command: sudo so-pcap-export "(host 10.0.2.10) and (port 4444) and (host 10.0.2.20) and (port 49678)" 479970930030821
Command output: reading from file /dev/stdin, link-type EN10MB (Ethernet), snapshot length 65536

If successful, the output was written to: /nsm/pcapout/479970930030821.pcap

PCAP file ./filtred_pcap_479970930030821.pcap retrieved successfully.
Executed command: sudo so-pcap-export "(host 10.0.2.10) and (port 59476) and (host 10.0.2.20) and (port 5802)" 1497695079384002
Command output: reading from file /dev/stdin, link-type EN10MB (Ethernet), snapshot length 65536

If successful, the output was written to: /nsm/pcapout/1497695079384002.pcap

PCAP file ./filtred_pcap_1497695079384002.pcap retrieved successfully.
Executed command: sudo so-pcap-export "(host 10.0.2.10) and (port 46816) and (host 10.0.2.20) and (port 1433)" 431196717720327
Command output: reading from file /dev/stdin, link-type EN10MB (Ethernet), snapshot length 65536

If successful, the output wa